# Model Formulation

This notebook can be used to run initial model with DCOPF and a basic budget variable. Before running this nodebook please refer to `Test-System-Data-Processing.ipynb` to preprocess generator, lines, buses, and loads data. 

In [168]:
using JuMP, Gurobi
using DataFrames, CSV

include(joinpath("capex_model", "expansion_tools.jl"))
include(joinpath("capex_model", "model.jl"))

expansion (generic function with 1 method)

In [176]:
length(1:ncol(loads))

240

In [ ]:
#run_model(load, basic_capex, save, "san_diego_system", "test_senario")
(buses, lines, gens, loads, variability, P, W) = load("san_diego_system")
model = expansion(buses, lines, gens, loads, variability, P, W)
#save(model, "san_diego_system", "psps_100", buses, lines, gens, loads);

Set parameter Username
Set parameter LicenseID to value 2669913
Academic license - for non-commercial use only - expires 2026-05-22
Model init okay.
Max constraints okay.
Storage constraints okay.


In [166]:
save(model, "san_diego_system", "nom_100", buses, lines, gens, loads);

In [152]:
sleep(1)